# Turkish ULMFiT from scratch

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [2]:
bs=128
torch.cuda.set_device(2)
data_path = Config.data_path()

lang = 'tr'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)

In [3]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
lm_fns = [mdl_path/f'{lang}_wt', mdl_path/f'{lang}_wt_vocab']

## Turkish wikipedia model

In [4]:
from nlputils import split_wiki,get_wiki

get_wiki(path,lang)
!head -n4 {path}/{name}

/home/jhoward/.fastai/data/trwiki/trwiki already exists; not downloading
<doc id="10" url="https://tr.wikipedia.org/wiki?curid=10" title="Cengiz Han">
Cengiz Han

Cengiz Han ("Cenghis Khan", "Çinggis Haan" ya da doğum adıyla Temuçin (anlamı: demirci), Moğolca: "Чингис Хаан" ya da "Tengiz" (anlamı: deniz), ; d. 1162 – ö. 18 Ağustos 1227), Moğol komutan, hükümdar ve Moğol İmparatorluğu'nun kurucusudur. Cengiz Han, 13. Yüzyılın başında Orta Asya'daki tüm göçebe bozkır kavimlerini birleştirerek bir ulus haline getirdi ve o ulusu "Moğol" siyasi kimliği çatısı altında topladı. Dünya tarihinin en büyük askeri dehalarından biri olarak kabul edilen Cengiz Han, hükümdarlığı döneminde 1206-1227 arasında Kuzey Çin'deki Batı Xia ve Jin Hanedanı, Türkistan'daki Kara Hıtay, Maveraünnehir, Harezm, Horasan ve İran'daki Harzemşahlar ile Kafkasya'da Gürcüler, Deşt-i Kıpçak'taki Rus Knezlikleri ve Kıpçaklar ile İdil Bulgarları üzerine gerçekleştirilen seferler sonucunda Pasifik Okyanusu'ndan Hazar Denizi’

In [5]:
dest = split_wiki(path,lang)

/home/jhoward/.fastai/data/trwiki/docs already exists; not splitting


Turkish is an [Agglutinative_language](https://en.wikipedia.org/wiki/Agglutinative_language) so it needs special care!

![Turkish morphemes example](images/turkish.jpg)

In [5]:
data = (TextList.from_folder(dest, processor=[OpenFileProcessor(), SPProcessor()])
        .split_by_rand_pct(0.1, seed=42)
        .label_for_lm()
        .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

In [ ]:
data = load_data(f'{lang}_databunch', bs=bs)

In [12]:
data.show_batch()

idx,text
0,"▁ele ▁geçirerek ▁politik ▁bir ▁özerklik ▁verdiğini ▁belirtmek le ▁birlikte ▁xxmaj ▁tibet ' in ▁yalnızca ▁1913 - 1950 ▁yılları ▁arasında ▁xxmaj ▁çin ' in ▁politik ▁nüfuz undan ▁çıktığını , ▁bölgenin ▁tarihi ▁olarak ▁xxmaj ▁çin ' e ▁ait ▁olduğunu ▁düşünmektedir . ▁xxmaj ▁tibet ' in ▁kendi ▁kültür ▁ve ▁zenginlik lerinin ▁"" kültürel ▁bir ▁soykırım "" a ▁tabi ▁tutulduğu ▁da ▁iddialar ▁arasındadır . ▁xxmaj ▁çin ▁hükümeti ▁ise ▁bu ▁"" kültürel ▁soykırım "" ▁iddia"
1,"▁dışı ▁kalan ▁olmasa ▁da , ▁o ▁yarışta ▁lastik ▁üreticisi ▁xxmaj ▁michel in ' le ▁yaşanan ▁problemler den ▁dolayı ▁güvenlik ▁gerekçesiyle ▁xxmaj ▁michel in ▁lastik leri ▁kullanan ▁tüm ▁takımlar ▁çekilmiş , ▁yarış a ▁sadece ▁6 ▁pilot ▁katılmıştır . ▁xxmaj ▁bunların ▁dışında ▁xxmaj ▁ ant ô ni o ▁xxmaj ▁pi zz onia ▁xxmaj ▁i ̇ talya ▁ve ▁sonrasında ▁sezonun ▁geri ▁kalanında ▁xxmaj ▁nick ▁xxmaj ▁he id feld ' in ▁yerine ▁yarışmıştır ."
2,"iye ▁oyunlarından ▁çok ▁farklı ▁bulduğunu ▁belirterek ▁taz iye yi ▁de ▁içinde ▁barındıran ▁benzersiz ▁bir ▁yapıt ▁olduğunu ▁yazdı . ▁< ▁/ ▁doc > ▁xxbos ▁xxmaj ▁anahtar sız ▁şifreleme ▁xxmaj ▁anahtar sız ▁şifreleme , ▁anahtar ▁kullanmaya n ▁kriptografik ▁algoritma lar , ▁veya ▁diğer ▁adlarıyla ▁xxmaj ▁veri ▁xxmaj ▁bütünlüğü ▁ve ▁xxmaj ▁özet ▁xxmaj ▁fonksiyonları , ▁veri ▁bütünlüğünü ▁garanti ▁etmek ▁için ▁kullanılan ▁xxup ▁md 5 , ▁xxup ▁sha -1 , ▁xxup ▁ rip em"
3,"rib ine ▁bağlı ▁olarak ▁akarsu ▁havza larında ▁artan ▁sediman ▁üretimi ▁f lü v yal ▁sistemlerde ▁etkili ▁olmuştur . ▁20 ▁yy ’ ın ▁ikinci ▁yarısından ▁itibaren ▁akarsular ▁üzerinde ▁çok ▁sayıda ▁baraj ▁yapılmıştır . ▁xxmaj ▁baraj ▁yapımı ▁nedeniyle , ▁akarsu nun ▁taşıdığı ▁sediman ▁miktarı ▁çevre nin ▁tahrip ▁edilmesi ▁nedeniyle ▁artmaktadır . ▁xxmaj ▁akarsu ▁yatakları ndan ▁kum ▁alınması , ▁akarsu nun ▁yatağı nın ▁yeniden ▁düzenlemesi ne ▁neden ▁olmaktadır . ▁xxmaj ▁teras lar ;"
4,"n ür lük "", ▁tek ▁başına ▁sensör ▁çözünürlüğü ne ▁bağlı ▁değildir . ▁"" mak sim um ▁sistem ▁çözünürlüğü "", ▁diğer ▁etkenler ▁( sen s ör ▁üzerinde ▁yer ▁alan ▁filtreler , ▁kullanılan ▁objektif in ▁optik ▁kalitesi ▁ve ▁diyafram ▁açıklığı ▁vb . ) ▁dışarıda ▁tutulur sa ▁aşağıdaki ▁formül ▁ile ▁hesaplanabilir ▁( birim ▁= ▁xxup ▁lp ▁/ ▁xxup ▁mm ▁xxmaj ▁i ̇ ng . : ▁ line ▁pa ir s ▁per ▁mili meter"


In [13]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.1, wd=0.1, pretrained=False).to_fp16()

In [14]:
lr = 3e-3
lr *= bs/48  # Scale learning rate by batch size

In [15]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.337346,4.472464,0.297502,10:04
1,4.145160,4.359449,0.304195,10:29
2,4.199796,4.386935,0.301542,10:29
3,4.132432,4.329662,0.305723,10:30
4,4.048324,4.241048,0.313223,10:32
5,3.943493,4.118834,0.323640,10:30
6,3.857112,3.969563,0.337475,10:30
7,3.695721,3.799551,0.354562,10:30
8,3.602813,3.652005,0.371855,10:31
9,3.519019,3.596706,0.379076,10:31


In [32]:
learn.to_fp32().save(lm_fns[0], with_opt=False)
learn.data.vocab.save(lm_fns[1].with_suffix('.pkl'))

## Turkish sentiment analysis

https://www.win.tue.nl/~mpechen/projects/smm/

### Language model

In [6]:
path_clas = path/'movies'
path_clas.ls()

[PosixPath('/home/jhoward/.fastai/data/trwiki/movies/tr_polarity.neg'),
 PosixPath('/home/jhoward/.fastai/data/trwiki/movies/tr_polarity.pos'),
 PosixPath('/home/jhoward/.fastai/data/trwiki/movies/tmp'),
 PosixPath('/home/jhoward/.fastai/data/trwiki/movies/models'),
 PosixPath('/home/jhoward/.fastai/data/trwiki/movies/tr_data_lm')]

In [7]:
pos = (path_clas/'tr_polarity.pos').open(encoding='iso-8859-9').readlines()
pos_df = pd.DataFrame({'text':pos})
pos_df['pos'] = 1
pos_df.head()

,text,pos
0,gerçekten harika bir yapim birçok kez izledim ...,1
1,her izledigimde hayranlik duydugum gerçek klas...,1
2,gerçekten tarihi savas filmleri arasinda tarti...,1
3,aldigi ödülleri sonuna dek hak eden muhtesem b...,1
4,özgürlük denilince aklima gelen ilk film.bir b...,1


In [8]:
neg = (path_clas/'tr_polarity.neg').open(encoding='iso-8859-9').readlines()
neg_df = pd.DataFrame({'text':neg})
neg_df['pos'] = 0
neg_df.head()

,text,pos
0,giseye oynayan bir film.mel gibson'in oyunculu...,0
1,bircok yonden sahip olduklari zayifliklari pop...,0
2,"1995 ten bu yana bu tür filmler artti , o zama...",0
3,mel gibson tam bir ingiliz düsmani her filmind...,0
4,milliyetçi bir film tavsiye etmiyorum.... \n,0


In [9]:
df = pd.concat([pos_df,neg_df], sort=False)

In [11]:
data_lm = (TextList.from_df(df, path_clas, cols='text', processor=[
    OpenFileProcessor(), SPProcessor.load(dest)], vocab=data.vocab)
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

data_lm.save(f'{lang}_clas_databunch')

In [ ]:
data_lm = load_data(f'{lang}_clas_databunch', bs=bs)

In [21]:
data_lm.show_batch()

idx,text
0,"bl il r = ) . ▁xxbos ▁haftada ▁bir ▁bu ▁filme ▁bak i yorum . . . ▁yorum ▁ya pil cak ▁bi ▁film ▁ di il ▁çünkü ▁mükemmel . ▁is le digi ▁as k ▁konusu yla , ▁özgürlük ▁konusu yla , ▁intikam ▁ve ▁ hir si yla ▁ve ▁tabi ki ▁ müz ig iyle ▁mükemmel ▁bir ▁film ▁ol mus . . ▁the ▁best ▁film ▁on ▁the ▁world ▁for ▁me ."
1,"▁benim ▁göz ya s lar im ▁olur ▁her ▁defa si nda ▁xxrep ▁4 ▁ . ▁san i rim ▁izlemeye n ▁yoktur ▁fazla ▁bir ▁söz ▁istemez . . ▁mü this ! !! . ▁xxbos ▁hayati min ▁filmi ▁ di yebilir im . ▁10 ▁numara ▁bir ▁film . ▁ele sti re lere ▁kap ali ▁bir ▁film ▁olma li . ▁çünkü ▁kötü ▁bir ▁yan ▁göre miyorum . . ▁xxbos ▁özgür ▁olma yi ,"
2,▁i sk ence ▁edilerek ▁idam ▁edilmesi . . . ve ▁sonunda ▁özgürlük ▁diye ▁hay kir isi . . . ha lan ▁unut a miyorum ▁xxrep ▁4 ▁ . ▁xxbos ▁ilk ▁bu ▁filmi ▁sinemada ▁izledi m ▁ve ▁insan in ▁inan di ktan ▁sonra ▁ ne leri ▁yap abi le ce gi ni ▁fark ▁etti m . ▁gerçekten ▁süper ▁film di . ▁halen ▁içi m den ▁geldi kçe ▁takip ▁izleri m ▁ve
3,▁al dan ip ta ▁filmi ▁izlemek ten ▁vazgeçme yin ▁xxrep ▁4 ▁ . ▁xxbos ▁harika ▁bir ▁film di ▁xxrep ▁5 ▁ . ▁xxbos ▁mükemmel ▁ötesi . . ▁ . ▁xxbos ▁hiç ▁ a bart miyorum ▁hayat im da ▁izledi gi m ▁en ▁iyi ▁filmlerden ▁biri ▁ di yebilir im . tam ▁bir ▁bas ya pit ▁nite ligi nde . o scar ▁al digi na ▁hiç ▁ sa si rma dim
4,". ▁herkes ▁izleme li . . . ▁xxbos ▁tek ▁kelime yle ▁bas ▁ya pit , ta ran tino ▁o ▁bir ▁dahi . iste ▁ tü t k ▁sinema si ▁bölgesel ▁ya da ▁yerel ▁drama lardan , mel od ram lardan ▁kurtul up , bir az cik ▁tarantino ▁kurgusu nu ▁ve ▁esp iri ▁an la yi sini ▁kavrama li . . ▁xxbos ▁müzikleri ▁konusu ▁ di y ologlar i ▁ki sa"


In [ ]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0, wd=0.1)

In [125]:
lr = 1e-3
lr *= bs/48

In [126]:
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.760790,4.130376,0.308103,00:07


In [127]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(5, slice(lr/10,lr*10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.134881,3.976427,0.329353,00:09
1,4.023767,3.897979,0.341518,00:09
2,3.788281,3.828589,0.353906,00:09
3,3.539453,3.817533,0.360067,00:09
4,3.315806,3.832252,0.362835,00:10


In [128]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Classifier

In [17]:
data_clas = (TextList.from_df(df, path_clas, cols='text', processor=[
    OpenFileProcessor(), SPProcessor.load(dest)], vocab=data_lm.vocab)
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='pos')
    .databunch(bs=bs, num_workers=1))

In [18]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False, wd=0.1).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [19]:
lr=2e-2
lr *= bs/48

In [20]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.460636,0.599994,0.744841,00:02
1,0.420206,0.548175,0.749531,00:02


In [154]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.430784,0.421478,0.821764,00:02
1,0.417428,0.405673,0.824578,00:02


In [155]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.358748,0.409607,0.840525,00:02
1,0.292855,0.343744,0.867730,00:02


In [156]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.265067,0.358663,0.867730,00:03
1,0.195801,0.361688,0.885553,00:03


In [157]:
learn_c.unfreeze()
learn_c.fit_one_cycle(4, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.120692,0.407334,0.887430,00:03
1,0.102027,0.458588,0.885553,00:03
2,0.061737,0.551036,0.885553,00:03
3,0.040430,0.583547,0.890244,00:04


Accuracy in Gezici (2018), *Sentiment Analysis in Turkish* is: `75.16%`.

In [158]:
learn_c.save(f'{lang}clas')

## fin